In [1]:
# Database
import sqlite3

# Machine learning
import torch
from torch import nn
import numpy as np
from sklearn.model_selection import train_test_split

# NLP
import spacy

# Own fns
import preprocessing


# 0. Fetch data from db

In [2]:
con = sqlite3.connect("../../db/db_training.sqlite")

cur = con.cursor()

In [3]:
# TODO: Use sqlutils.get_sql_statement()
with open("../../db/sql/select_all_text.sql") as sql_file:
    sql = sql_file.read()
    sql_file.close()

cur.execute(sql)

'''
Response format: list[(text, category)]
'''
response = cur.fetchall()

In [4]:
data_dict = {}
'''
Format: {[category]: list[page_text_content]}
'''

for page in response:
    text = page[0]
    category = page[1]

    if category not in list(data_dict.keys()):
        data_dict[category] = [text]
    else :
        data_dict[category].append(text)

print(len(data_dict["0"]))

22


# 1. Cleaning

In [5]:
cleaned_data_dict = {}
nlp = spacy.load("fr_core_news_lg")

for category in list(data_dict.keys()):
    cleaned_data_dict[category] = []

for category in list(data_dict.keys()):
    for text_content in data_dict[category]:
        cleaned_text = preprocessing.full_cleaning(text_content, nlp)

        cleaned_data_dict[category].append(cleaned_text)

print(cleaned_data_dict)

{'2': ['hitachi inspirer the next smart tv être ref hk hdr cm tcl tcl smart tv ref tcl dolby hdr atmos cm google tv integre lg lg cut smart tv être ref ur ic processeur avoir al gen hdr cm airplay dl integre adams paiement frais notre prix eco taxe inclure', 'lg lg good systeme audio high power cm de hauteur bluetooth boomer eclairage multicolore fonction dj et karaoke lg enceinte mobile lg eco participation inclure profiter de bas ultra puissant pour votre soiree de surpuissant double bass boost le haut parleur de bas lumineux shiny et dautre fonctionnalite apporter un rythme puissant pour un soiree memorabl votre smartphone se mettre au rythme de le musique illuminer le fete vous pouvoir connecter jusqua trois smartphone en mem temps et le tenir en air ou en placer un dans le socle pour voir le flash au rythme de le musique ea conforamer an de garantie sur electromenager joon super grave', 'smart tv road ex le smart tv oo led full hd cm smart tv wifi netflix eco part full hd hdmi th 

# 2. Create a vocabulary

In [6]:
all_text = ""

for category in list(cleaned_data_dict.keys()):
    temp_text = " ".join(cleaned_data_dict[category])
    print(temp_text)

    if category == list(cleaned_data_dict.keys())[0]:
        all_text += temp_text
    else:
        all_text += " " + temp_text

# TODO: Fix "'" should not be in the vocabulary ! Check preprocessing.clean_text
vocabulary = np.unique(all_text.split(" "))
print(list(vocabulary))
print(len(list(vocabulary)))

hitachi inspirer the next smart tv être ref hk hdr cm tcl tcl smart tv ref tcl dolby hdr atmos cm google tv integre lg lg cut smart tv être ref ur ic processeur avoir al gen hdr cm airplay dl integre adams paiement frais notre prix eco taxe inclure lg lg good systeme audio high power cm de hauteur bluetooth boomer eclairage multicolore fonction dj et karaoke lg enceinte mobile lg eco participation inclure profiter de bas ultra puissant pour votre soiree de surpuissant double bass boost le haut parleur de bas lumineux shiny et dautre fonctionnalite apporter un rythme puissant pour un soiree memorabl votre smartphone se mettre au rythme de le musique illuminer le fete vous pouvoir connecter jusqua trois smartphone en mem temps et le tenir en air ou en placer un dans le socle pour voir le flash au rythme de le musique ea conforamer an de garantie sur electromenager joon super grave smart tv road ex le smart tv oo led full hd cm smart tv wifi netflix eco part full hd hdmi th le smart tv le

# 3. Feature extraction (BOW)

In [7]:
X = []
y = []

for category in list(cleaned_data_dict.keys()):
    for text in cleaned_data_dict[category]:
        X.append(preprocessing.get_bags_of_words_vector(text, vocabulary))
        y.append(int(category)) # category should be integer from db

print(len(X))
print(len(y))

66
66


# 4. Model

In [8]:
class PageClassifier(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=128, dtype=torch.float32),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=output_features, dtype=torch.float32),
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

# 5. Training

In [9]:
def accuracy_fn(y_true, y_pred):
    correct = (
        torch.eq(input=y_true, other=y_pred).sum().item()
    )  # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100  # Calcul simple de pourcentage
    return acc

In [10]:
def training_loop(PageClassifier, X, y):

    # Init model
    torch.manual_seed(42)
    model = PageClassifier(input_features=len(X[0]), output_features=len(list(set(y))))

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

    # Prepare data
    X = torch.tensor(X).type(torch.float32)
    y = torch.tensor(y).type(torch.long)

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Training loop
    torch.manual_seed(42)
    epochs = 10

    for epoch in range(epochs + 1):
        """
        Train
        """
        model.train()

        # 1. Forward pass
        y_logits = model(X_train)
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

        # 2. Metrics
        loss = loss_fn(y_logits, y_train)
        acc = accuracy_fn(y_true=y_train, y_pred=y_pred)

        # 2.1 Save metrics
        # loss_history.append(loss.cpu().detach().numpy())
        # acc_history.append(acc)

        # 3. Zero Grad
        optimizer.zero_grad()

        # 4. Backpropagation
        loss.backward()

        # 5. Optimmizer step
        optimizer.step()

        """
        Validation
        """
        model.eval()

        with torch.inference_mode():
            # 1. Forward pass
            y_val_logits = model(X_val)
            y_val_pred = torch.softmax(y_val_logits, dim=1).argmax(dim=1)

            # 2. Metrics
            val_loss = loss_fn(y_val_logits, y_val)
            val_acc = accuracy_fn(y_pred=y_val_pred, y_true=y_val)

            # 2.1 Save metrics
            # test_loss_history.append(test_loss.cpu().detach().numpy())
            # test_acc_history.append(test_acc)

        # Print out what's happening
        # if epoch % 25 == 0:
        print(
            f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {val_loss:.5f}, Test Acc: {val_acc:.2f}%"
        )

    return model

In [11]:
model = training_loop(PageClassifier, X, y)

Epoch: 0 | Loss: 1.13381, Acc: 21.15% | Test Loss: 8.30076, Test Acc: 57.14%
Epoch: 1 | Loss: 6.23695, Acc: 69.23% | Test Loss: 0.09344, Test Acc: 92.86%
Epoch: 2 | Loss: 0.01577, Acc: 98.08% | Test Loss: 0.00001, Test Acc: 100.00%
Epoch: 3 | Loss: 0.04965, Acc: 98.08% | Test Loss: 0.00171, Test Acc: 100.00%
Epoch: 4 | Loss: 0.00011, Acc: 100.00% | Test Loss: 0.20713, Test Acc: 92.86%
Epoch: 5 | Loss: 0.00000, Acc: 100.00% | Test Loss: 1.37052, Test Acc: 85.71%
Epoch: 6 | Loss: 0.00000, Acc: 100.00% | Test Loss: 2.33615, Test Acc: 85.71%
Epoch: 7 | Loss: 0.00000, Acc: 100.00% | Test Loss: 3.21711, Test Acc: 85.71%
Epoch: 8 | Loss: 0.00000, Acc: 100.00% | Test Loss: 4.05122, Test Acc: 85.71%
Epoch: 9 | Loss: 0.00000, Acc: 100.00% | Test Loss: 4.83873, Test Acc: 85.71%
Epoch: 10 | Loss: 0.00000, Acc: 100.00% | Test Loss: 5.58082, Test Acc: 85.71%


# 5. Prediction

In [12]:
def predict(model: PageClassifier, input: list[int], label_mapping: dict[str]):
    model.eval()

    with torch.inference_mode():
        X = torch.tensor([input]).type(torch.float32)
        y_logits = model(X)
        y_softmax = torch.softmax(y_logits, dim=1)
        y_pred = y_softmax.argmax(dim=1)
        
        result = y_softmax.detach().numpy()
        class_prediction = label_mapping[y_pred.detach().numpy()[0]]

        print("result", result)
        print("class_prediction =>", class_prediction)

In [13]:
label_mapping = {0: "Meuble | Aménagement int | Décoration", 1:"Électroménager  | Ustensiles", 2: "Multimédia (son,TV)"}

# TODO: Use real test data
input = preprocessing.pipeline_from_raw_text_to_vectors(data_dict['1'][-1],nlp, vocabulary)
print(input)
predict(model, input, label_mapping)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 5, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

TODO:
- Try auto-sklearn
- Try lazy predict
- Create model.py (with pytorch)